# 车辆分类

使用迁移学习进行车辆分类

In [7]:
#安装 Paddlehub
! pip install paddlehub

^C


You should consider upgrading via the 'D:\venvs\pdpd-gpu\Scripts\python.exe -m pip install --upgrade pip' command.


  Using cached tqdm-4.61.2-py2.py3-none-any.whl (76 kB)
  Using cached PyYAML-5.4.1-cp39-cp39-win_amd64.whl (213 kB)
  Using cached filelock-3.0.12-py3-none-any.whl (7.6 kB)
  Using cached click-8.0.1-py3-none-any.whl (97 kB)
  Using cached distlib-0.3.2-py2.py3-none-any.whl (338 kB)
  Using cached scikit_learn-0.24.2-cp39-cp39-win_amd64.whl (6.9 MB)
  Using cached threadpoolctl-2.2.0-py3-none-any.whl (12 kB)
  Using cached joblib-1.0.1-py3-none-any.whl (303 kB)
  Using cached scipy-1.7.0-cp39-cp39-win_amd64.whl (33.8 MB)
  Created wheel for future: filename=future-0.18.2-py3-none-any.whl size=491070 sha256=b52fab8cd1f8d508566f9a6d8f066ed03f20e2f2993192651c9100fa189768a0
  Stored in directory: c:\users\rswor\appdata\local\pip\cache\wheels\2f\a0\d3\4030d9f80e6b3be787f19fc911b8e7aa462986a40ab1e4bb94
  Created wheel for easydict: filename=easydict-1.9-py3-none-any.whl size=6361 sha256=692efb35734ea45a2eae65be1c41ef6acd3740071515d966168129ca4a4f34f5
  Stored in directory: c:\users\rswor\ap

You should consider upgrading via the 'D:\venvs\pdpd-gpu\Scripts\python.exe -m pip install --upgrade pip' command.


加载数据

In [1]:
import paddlehub.vision.transforms as T
from car_classification_dataset import CarsForClassification
transforms = T.Compose(
        [T.Resize((256, 256)),
         T.CenterCrop(224),
         T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])],
        to_rgb=True)

cars_train = CarsForClassification(transforms)
cars_test =  CarsForClassification(transforms, mode='test')

In [2]:
CarsForClassification()

In [2]:
import paddlehub as hub
import json
from car_classification_dataset import CAR_CLASSIFICATION_PATH

with open(CAR_CLASSIFICATION_PATH + "map.json") as f:
    dat = json.load(f)
model = hub.Module(name="resnet50_vd_imagenet_ssld", label_list=list(dat.keys()))

load pretrained parameters success


In [3]:
config = hub.RunConfig(
    use_cuda=True,                              #是否使用GPU训练，默认为False；
    num_epoch=10,                                #Fine-tune的轮数；
    checkpoint_dir="car_classification_model",#模型checkpoint保存路径, 若用户没有指定，程序会自动生成；
    batch_size=128,                              #训练的批大小，如果使用GPU，请根据实际情况调整batch_size；
    eval_interval=50,                           #模型评估的间隔，默认每100个step评估一次验证集；
    )  #Fine-tune优化策略；

In [4]:
import paddle
from paddlehub.finetune.trainer import Trainer
optimizer = paddle.optimizer.Adam(learning_rate=0.001, parameters=model.parameters())
trainer = Trainer(model, optimizer, checkpoint_dir='car_classification_model', use_gpu=False)

[2021-07-25 11:19:04,043] [ WARNING] - PaddleHub model checkpoint not found, start from scratch...


In [ ]:
trainer.train(cars_train, epochs=10, batch_size=64, eval_dataset=cars_test, save_interval=1)

Error: Session cannot generate requests